# START OF COMMON CODE

In [1]:
POLYGON_PATH = 'https://raw.githubusercontent.com/IsamAljawarneh/datasets/1c2a6af7dea7aa93105ac1d1d0118d07bd681d8a/data/nyc_polygon.geojson'
POINTS_PATH  = 'https://raw.githubusercontent.com/IsamAljawarneh/datasets/1c2a6af7dea7aa93105ac1d1d0118d07bd681d8a/data/NYC_Pilot2_PM_Part1.csv'

In [2]:
from collections import defaultdict
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
from typing import Callable, List, Union, Any, Dict

In [3]:
def load_polygons(polygons_geojson: str) -> gpd.GeoDataFrame:
    try:
        polygons = gpd.read_file(polygons_geojson)
        if polygons.empty:
            raise ValueError("GeoJSON file contains no data.")
        return polygons
    except Exception as e:
        raise IOError(f"Failed to load polygons from GeoJSON: {e}")

In [4]:
def polygons_find_covering_geocodes(
    polygons: gpd.GeoDataFrame,
    polygon_to_geocodes: Callable[[Polygon, int], List[str]],
    geocode_resolution: int
):
    polygons['covering_geocodes'] = polygons['geometry'].apply(lambda x: polygon_to_geocodes(x, geocode_resolution))

In [5]:
def load_polygons_with_covering_geocodes(
    polygons_geojson: str,
    polygon_to_geocodes: Callable[[Polygon, int], List[str]],
    geocode_resolution: int
) -> gpd.GeoDataFrame:
    polygons = load_polygons(polygons_geojson)
    polygons_find_covering_geocodes(polygons, polygon_to_geocodes, geocode_resolution)
    return polygons

In [6]:
def create_geocode_lookup(polygons: gpd.GeoDataFrame) -> Dict[str, List[int]]:
    geocode_lookup = defaultdict(list)
    for idx, codes in polygons['covering_geocodes'].items():
        for geocode in codes:
            geocode_lookup[geocode].append(idx)

    return geocode_lookup

In [7]:
def load_points(points_csv: str, crs: Any = "EPSG:4326") -> gpd.GeoDataFrame:
    try:
        df = pd.read_csv(points_csv)
        if df.empty:
            raise ValueError("CSV file contains no data.")
        if not {'latitude', 'longitude'}.issubset(df.columns):
            raise ValueError("CSV must contain 'latitude' and 'longitude' columns.")
        gdf = gpd.GeoDataFrame(
            df, 
            geometry=[Point(xy) for xy in zip(df.longitude, df.latitude)],
            crs=crs
        )
        return gdf
    except Exception as e:
        raise IOError(f"Failed to load points from CSV: {e}")

In [8]:
def points_find_geocode(
    points: gpd.GeoDataFrame,
    point_to_geocode: Callable[[float, float, int], str],
    geocode_resolution: int
):
    points['geocode'] = points.apply(
        lambda row: point_to_geocode(row['latitude'], row['longitude'], geocode_resolution),
        axis=1
    )

In [9]:
def load_points_with_geocode(
    points_csv: str,
    point_to_geocode: Callable[[float, float, int], str],
    geocode_resolution: int,
    crs: Any = "EPSG:4326"
) -> gpd.GeoDataFrame:
    gdf = load_points(points_csv, crs)
    points_find_geocode(gdf, point_to_geocode, geocode_resolution)
    return gdf

In [10]:
def spatial_join_with_geocodes(
    polygons: gpd.GeoDataFrame,
    geocode_lookup: Dict[str, List[int]],
    points: gpd.GeoDataFrame
) -> gpd.GeoDataFrame:
    # Create a copy
    points = points.copy()

    # Filter step: find potential matches
    points['potential_matches'] = points['geocode'].map(geocode_lookup)

    # Refinement step: verify actual containment
    def verify_containment(point: pd.Series, candidate_polygons: Union[List[int], float]) -> int:
        if isinstance(candidate_polygons, float):
            # assert pd.isna(candidate_polygons)
            return -1
        if not candidate_polygons:
            return -1
        point_geom = point['geometry']
        for index in candidate_polygons:
            polygon = polygons.loc[index, 'geometry']
            if point_geom.within(polygon):
                return index
        return -1

    # Apply refinement
    points['polygon_index'] = points.apply(lambda row: verify_containment(row, row['potential_matches']), axis=1)

    return points

# START OF H3-SPECIFIC CODE

## geojson2h3 Python module

In [11]:
import h3

In [13]:
FEATURE = 'Feature'
FEATURE_COLLECTION = 'FeatureCollection'
POLYGON = 'Polygon'
MULTI_POLYGON = 'MultiPolygon'


def flatten(arrays):
    """
    Utility for efficient flattening of arrays. This mutates input,
    flattening into the first array in the list.
    """
    out = None
    for arr in arrays:
        if out is not None:
            out.extend(arr)
        else:
            out = arr
            if not isinstance(out, list):
                out = list(out)
    return list(set(out))


def centroid(polygon):
    """
    Utility to compute the centroid of a polygon.
    """
    lng_sum = 0
    lat_sum = 0
    count = 0
    loop = polygon[0]
    for point in loop:
        lng_sum += point[0]
        lat_sum += point[1]
        count += 1
    return lng_sum / count, lat_sum / count


def feature_collection_to_h3_set(feature_collection, resolution):
    """
    Convert a GeoJSON feature collection to a set of hexagons.
    """
    features = feature_collection.get('features')
    if not features:
        raise ValueError('No features found')
    return flatten(list(map(lambda feature: feature_to_h3_set(feature, resolution), features)))


def feature_to_h3_set(feature, resolution, ensure_output=False):
    """
    Convert a GeoJSON feature to a set of hexagons.
    """
    type = feature.get('type')
    geometry = feature.get('geometry')
    geometry_type = None if geometry is None else geometry.get('type')

    if type == FEATURE_COLLECTION:
        return feature_collection_to_h3_set(feature, resolution)

    if type != FEATURE:
        raise ValueError(f"Unhandled type: {type}")

    if geometry_type not in (POLYGON, MULTI_POLYGON):
        raise ValueError(f"Unhandled geometry type: {geometry_type}")

    # Ensure the correct format for the polygon argument
    if geometry_type == POLYGON:
        polygons = [geometry['coordinates']]
    else:  # MULTI_POLYGON
        polygons = geometry['coordinates']

    def process(polygon):
        result = h3.polyfill({'type': 'Polygon', 'coordinates': polygon}, resolution, True)
        if result or not ensure_output:
            return result

        # If we got no results, index the centroid
        lng, lat = centroid(polygon)
        return [h3.geo_to_h3(lat, lng, resolution)]

    # Polyfill each polygon and flatten the results
    return flatten(
        list(map(process, polygons))
    )


def h3_to_feature(h3_index, properties=None):
    """
    Convert a single H3 hexagon to a `Polygon` feature.
    """
    coordinates = [h3.h3_to_geo_boundary(h3_index, True)]
    return {
        "type": FEATURE,
        "id": h3_index,
        "properties": properties if properties else {},
        "geometry": {
            "type": POLYGON,
            "coordinates": coordinates
        }
    }


def h3_set_to_feature(hexagons, properties=None):
    """
    Convert a set of hexagons to a GeoJSON `Feature` with the set outline(s).
    """
    polygons = h3.h3_set_to_multi_polygon(hexagons, True)
    is_multi_polygon = len(polygons) > 1
    geometry_type = MULTI_POLYGON if is_multi_polygon else POLYGON
    coordinates = polygons if is_multi_polygon else polygons[0] if polygons else []
    return {
        "type": FEATURE,
        "properties": properties if properties else {},
        "geometry": {
            "type": geometry_type,
            "coordinates": coordinates
        }
    }


def h3_set_to_multi_polygon_feature(hexagons, properties=None):
    """
    Convert a set of hexagons to a GeoJSON `MultiPolygon` feature with the
    outlines of each individual hexagon.
    """
    coordinates = list(map(
        lambda h3Index: [h3.h3ToGeoBoundary(h3Index, True)],  # Wrap in an array for a single-loop polygon
        hexagons
    ))
    return {
        "type": FEATURE,
        "properties": properties if properties else {},
        "geometry": {
            "type": MULTI_POLYGON,
            "coordinates": coordinates
        }
    }


def h3_set_to_feature_collection(hexagons, get_properties=None):
    """
    Convert a set of hexagons to a GeoJSON `FeatureCollection` with each hexagon
    in a separate `Polygon` feature with optional properties.
    """
    features = []
    for h3_index in hexagons:
        properties = get_properties(h3_index) if get_properties else {}
        features.append(h3_to_feature(h3_index, properties))
    return {
        "type": FEATURE_COLLECTION,
        "features": features
    }

## Remainder of geocovering code

In [15]:
def polygon_to_h3(polygon: Polygon, resolution: int) -> List[str]:
    feature = {
        "type": "Feature",
        "properties": {},
        "geometry": polygon.__geo_interface__
    }
    return feature_to_h3_set(feature, resolution, True)

In [16]:
def point_to_h3(latitude: float, longitude: float, resolution: int) -> str:
    return h3.geo_to_h3(latitude, longitude, resolution)

## Testing

In [17]:
# Load data
polygons = load_polygons_with_covering_geocodes(
    POLYGON_PATH,
    polygon_to_h3,
    8  # H3 resolution
)
points = load_points_with_geocode(
    POINTS_PATH,
    point_to_h3,
    8  # H3 resolution
)

In [18]:
# Create lookup dictionary
geocode_lookup = create_geocode_lookup(polygons)

In [19]:
# Using the spatial join function with H3
h3_results = spatial_join_with_geocodes(
    polygons,
    geocode_lookup,
    points
)

In [20]:
h3_results

,SensorID,time,latitude,longitude,bin0,bin1,bin2,bin3,bin4,bin5,...,bin21,bin22,bin23,temperature,humidity,pm25,geometry,geocode,potential_matches,polygon_index
0,NYCP2_CS01A,1631277304,40.847672,-73.869316,11,1,1,0,0,0,...,0,0,0,23.7,57.3,4.508813,POINT (-73.86932 40.84767),882a100133fffff,"[38, 292]",38
1,NYCP2_CS01A,1631277308,40.847668,-73.869316,22,4,1,0,0,2,...,0,0,0,23.7,57.8,5.462420,POINT (-73.86932 40.84767),882a100133fffff,"[38, 292]",38
2,NYCP2_CS01A,1631277313,40.847649,-73.869362,40,1,1,0,0,1,...,0,0,0,23.7,57.8,5.154881,POINT (-73.86936 40.84765),882a100133fffff,"[38, 292]",38
3,NYCP2_CS01A,1631277318,40.847649,-73.869362,26,1,0,0,0,0,...,0,0,0,23.6,57.6,4.508813,POINT (-73.86936 40.84765),882a100133fffff,"[38, 292]",38
4,NYCP2_CS01A,1631277323,40.847649,-73.869362,44,4,0,1,0,0,...,0,0,0,23.6,57.5,5.539503,POINT (-73.86936 40.84765),882a100133fffff,"[38, 292]",38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169994,NYCP2_CS03A,1631457109,40.823353,-73.890488,115,11,2,0,1,0,...,0,0,0,24.6,54.8,5.460360,POINT (-73.89049 40.82335),882a100a99fffff,[176],176
169995,NYCP2_CS03A,1631457114,40.823349,-73.890480,132,8,2,0,0,0,...,0,0,0,24.6,54.8,5.298209,POINT (-73.89048 40.82335),882a100a99fffff,[176],176
169996,NYCP2_CS03A,1631457119,40.823349,-73.890480,147,14,0,0,0,0,...,0,0,0,24.6,54.8,6.470661,POINT (-73.89048 40.82335),882a100a99fffff,[176],176
169997,NYCP2_CS03A,1631457124,40.823345,-73.890488,121,8,2,0,1,1,...,0,0,0,24.6,54.6,6.424142,POINT (-73.89049 40.82335),882a100a99fffff,[176],176


# Comparison with traditional spatial join

In [21]:
def spatial_join_traditional(
    polygons: gpd.GeoDataFrame,
    points: gpd.GeoDataFrame
) -> gpd.GeoDataFrame:
    # Create a copy
    points = points.copy()

    def find_containment(point: pd.Series) -> int:
        point_geom = point['geometry']
        for index, row in polygons.iterrows():
            polygon = row['geometry']
            if point_geom.within(polygon):
                return index
        return -1

    points['polygon_index'] = points.apply(find_containment, axis=1)

    return points

In [22]:
points_baseline = load_points(POINTS_PATH)
polygons_baseline = load_polygons(POLYGON_PATH)

In [23]:
# baseline_results = gpd.sjoin(points_baseline, polygons_baseline, how="inner", predicate='within')
# baseline_results['polygon_index'] = baseline_results['index_right']

baseline_results = spatial_join_traditional(polygons_baseline, points_baseline)

In [24]:
def evaluate_results(test_results: gpd.GeoDataFrame, baseline_results: gpd.GeoDataFrame, method_name):
    # Aligning indices for comparison
    test_results = test_results[['polygon_index']].copy()
    baseline_results = baseline_results[['polygon_index']].copy()

    # print(test_results['polygon_index'][test_results['polygon_index'] == -1])

    # Join the test results with the baseline on the point index
    comparison_df = test_results.join(baseline_results, rsuffix='_baseline', how='inner')

    # Calculate the match rate
    match_rate = (comparison_df['polygon_index'] == comparison_df['polygon_index_baseline']).mean()
    
    print(f"Match Rate for {method_name}: {match_rate * 100}%")

In [26]:
evaluate_results(h3_results, baseline_results, f"H3 with resolution 8")

Match Rate for H3 with resolution 8: 87.23491867407866%
